# Code Objective:

*   BERT Model for Coarse Grained Evaluation
*   XLMRoBerta Model for Coarse Grained Evaluation
*   Saving Probabilities (BERT and XLMRoberta) for Ensemble

# Code Result:
*   Accuracy - BERT Model for Coarse Grained Evaluation = 91.63 %
*   Accuracy - XLMRoBerta for Coarse Grained Evaluation = 89.76 %





# Import Libraries

In [ ]:
!pip install transformers

In [ ]:
import pandas as pd
import numpy as np
import json, re
from tqdm import tqdm_notebook
from uuid import uuid4

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from transformers import glue_compute_metrics
from transformers import Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig, BertForSequenceClassification
from transformers import XLMRobertaTokenizer, XLMRobertaModel, XLMRobertaConfig, XLMRobertaForSequenceClassification

print("GPU Torch Available = {}".format(torch.cuda.is_available()))
print("Torch Version = {}".format(torch.__version__))

# Data Loading

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Loading Training Data
file = '/content/drive/My Drive/CONSTRAINT 2021 Projects (AAAI)/Hindi_Task/Dataset/NER_binary_train.xlsx'
train_df = pd.read_excel(file)
train_df

In [ ]:
# Loading Validation Data
file = '/content/drive/My Drive/CONSTRAINT 2021 Projects (AAAI)/Hindi_Task/Dataset/NER_binary_validate.xlsx'
test_df = pd.read_excel(file)
test_df

In [ ]:
# Data Preparation into Pandas Dataframe for Model Input
def get_data(a):
  Unique_ID = list(a['Unique ID'])
  sentence = list(a['Post'])
  text_labels = list(a['Labels Set'])

  label = []
  for i in text_labels:
    if i=='non-hostile':
        label.append(0)
    elif i=='hostile':
        label.append(1)

  raw_data_train = {'UID':Unique_ID,'sentence': sentence, 'label': label}
  df = pd.DataFrame(raw_data_train, columns = ['UID','sentence','label'])
  return df

train_data = get_data(train_df)
test_data  = get_data(test_df)


print(train_data[0:3])
print(test_data[0:3])

# Model Parameters

In [ ]:
# Choose and Load Model
model_name = 'Bert'

if (model_name == 'Bert'):
  # Bert Parameters
  config = BertConfig.from_pretrained('bert-base-multilingual-cased',num_labels=2)
  tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
  model = BertForSequenceClassification(config)
elif (model_name = 'Roberta'):
  # XLMRoberta Parameters
  config = XLMRobertaConfig.from_pretrained('xlm-roberta-base',num_labels=2)
  tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
  model = XLMRobertaForSequenceClassification(config)
else:
  print('Choose correct Model')

# Data Preparation for Model Input

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.sentence = dataframe.sentence
        self.targets = self.data.label
        self.max_len = max_len
    def __len__(self):
        return len(self.sentence)
    def __getitem__(self, index):
        sentence1 = str(self.sentence[index])

        inputs = self.tokenizer.encode_plus(sentence1,
                                            truncation=True,
                                            add_special_tokens=True,
                                            max_length=self.max_len,
                                            pad_to_max_length=True,
                                            return_token_type_ids=True)
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]

        return {'input_ids': torch.tensor(ids, dtype=torch.long),
                'attention_mask': torch.tensor(mask, dtype=torch.long),
                'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
                'labels': torch.tensor(self.targets[index], dtype=torch.long)
               }

In [ ]:
# Dataset for Input into Model
MAX_LEN = 128                                                # Max Sequence Length
training_set = CustomDataset(train_data, tokenizer, MAX_LEN) # Training Set
testing_set = CustomDataset(test_data, tokenizer, MAX_LEN)   # Validation Set

# Training and Evaluation Phase

In [ ]:
# Device Mapping Select (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.cuda()

# Training Arguments
training_args = TrainingArguments(output_dir="./models/model_name",
                                  overwrite_output_dir=True,
                                  do_train=True,
                                  do_eval=True,
                                  per_device_train_batch_size=28,
                                  per_device_eval_batch_size=28,
                                  num_train_epochs=20,
                                  logging_steps=100,
                                  logging_first_step=True,
                                  save_steps=0,
                                  evaluate_during_training=True)

# Metric for Performance Evaluation
def compute_metrics(p):
  preds = np.argmax(p.predictions, axis=1)
  return glue_compute_metrics("mnli", preds, p.label_ids)

# Trainer for training Model
trainer = Trainer(model = model,
                  args = training_args,
                  train_dataset = training_set,
                  eval_dataset = testing_set,
                  compute_metrics = compute_metrics)

In [ ]:
# Training Model
trainer.train()

In [ ]:
# Evaluation of Model on Validation Data
trainer.evaluate(testing_set)

# Trained Model Save and Load for later use

In [ ]:
# Model Save
model_save_path = '/content/drive/My Drive/CONSTRAINT 2021 Projects (AAAI)/Hindi_Task/Weights/XLMR_state_dict_NER_'
torch.save(model.state_dict(), model_save_path + str(uuid4()) + '.pth')

In [ ]:
# Model Load
model_load_path = '/content/drive/My Drive/CONSTRAINT 2021 Projects (AAAI)/Hindi_Task/Weights/XLMR_state_dict_cda2c4d1-e5e0-49a4-acc3-bdf39e74493c.pth'
model.load_state_dict(torch.load(model_load_path, map_location=device))

# Prediction

In [ ]:
# Prediction for a given 

def prepare_features(seq_1, max_seq_length = 128, zero_pad = False, include_CLS_token = True, include_SEP_token = True):
    ## Tokenzine Input
    tokens_a = tokenizer.tokenize(seq_1)

    ## Truncate
    if len(tokens_a) > max_seq_length - 2:
        tokens_a = tokens_a[0:(max_seq_length - 2)]
    ## Initialize Tokens
    tokens = []
    if include_CLS_token:
        tokens.append(tokenizer.cls_token)
    ## Add Tokens and separators
    for token in tokens_a:
        tokens.append(token)

    if include_SEP_token:
        tokens.append(tokenizer.sep_token)

    input_ids = tokenizer.convert_tokens_to_ids(tokens)
    ## Input Mask 
    input_mask = [1] * len(input_ids)
    ## Zero-pad sequence lenght
    if zero_pad:
        while len(input_ids) < max_seq_length:
            input_ids.append(0)
            input_mask.append(0)
    return torch.tensor(input_ids).unsqueeze(0), input_mask

def predict(text):
  model.eval()
  input_feature, _ = prepare_features(text)
  if torch.cuda.is_available():
    input_feature = input_feature.cuda()
  output = model(input_feature)[0]
  _, pred_label = torch.max(output.data, 1)
  prediction = pred_label
  return prediction

index = 13                           # Index for identifying post 
text = test_data['sentence'][index]  # Post as a string

pred = predict(text)                 # Prediction of Hostile vs Non-Hostile Class
print(pred) 

# Prediction Probabilities

In [ ]:
def predict_looper(text):
  model.eval()
  input_feature, _ = prepare_features(text)
  if torch.cuda.is_available():
    input_feature = input_feature.cuda()
  output = model(input_feature)[0]
  Softmax = nn.Softmax(dim=1)
  prob = Softmax(output)
  prob_cpu = prob.cpu().detach().numpy()
  return prob_cpu[0]

data_1 = train_data
data_2 = test_data

pred_1 = []
for i in range(len(data_1)):
  text = data_1['sentence'][i]
  pred_1.append(predict_looper(text))

pred_2 = []
for i in range(len(data_2)):
  text = data_2['sentence'][i]
  pred_2.append(predict_looper(text))

y1 = np.array(pred_1)
y2 = np.array(pred_2)

np.array(y1).dump(open('Train_Probs_Coarse.npy', 'wb')) # Training Probabilities
np.array(y2).dump(open('Test_Probs_Coarse.npy', 'wb'))  # Testing Probabilities